# Analysis of cell-free synthetic gene expression

In [ ]:
import flapjack
from flapjack import Flapjack, layout_print
import numpy as np
import plotly
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.io as io
import pandas as pd
import getpass
%matplotlib inline

In [ ]:
user = input()
passwd = getpass.getpass()
fj = Flapjack('flapjack.rudge-lab.org:8000')
fj.log_in(username=user, password=passwd)

## Figure 8B - Visualising raw data

As well as using the Flapjack webapp (https://github.com/SynBioUC/flapjack_frontend/wiki/CFPS-of-CFP-YFP-variants), you can obtain the figure 8B using the Flapjack Python package.

In [ ]:
study_id = fj.get('study', name='CFPS of CFP YFP variants').id
media_id = fj.get('media', name='E1').id

fig = fj.plot(study=study_id,
             media=media_id,
             normalize='None',
             subplots='Signal',
             markers='Vector',
             plot='Mean +/- std')
fig

In [ ]:
# Modify width and size (optional)
fig = flapjack.layout_print(fig, width=2, height=2, font_size=6)
fig.layout.annotations[0].text='CFP channel'
fig.layout.annotations[1].text='YFP channel'
fig

In [ ]:
fname = 'Figure8B.png'
io.write_image(fig, fname)

## Figure 8C - Computing and plotting reaction velocity
Reaction velocity is the first derivative of fluorescence (a time series).
As well as using the Flapjack webapp (https://github.com/SynBioUC/flapjack_frontend/wiki/CFPS-of-CFP-YFP-variants), you can obtain figure 8C using the Flapjack Python package.

### CFP

In [ ]:
study = fj.get('study', name='CFPS of CFP YFP variants')
vector_cfp_names = ['pCFP', 'pdeCFP']
vector_cfp_ids = [fj.get('vector', name=vec_name).id for vec_name in vector_cfp_names]
vector_yfp_names = ['pYFP', 'pdeYFP']
vector_yfp_ids = [fj.get('vector', name=vec_name).id for vec_name in vector_yfp_names]
cfp_id = fj.get('signal', name='CFP').id
yfp_id = fj.get('signal', name='YFP').id

In [ ]:
fig = fj.plot(study=study.id,
             vector=vector_cfp_ids,
             signal=cfp_id,
             type='Velocity',
             pre_smoothing=21,
             post_smoothing=21,
             normalize='Mean/std',
             subplots='Media',
             markers='Vector',
             plot='Mean'
             )
fig

In [ ]:
fig = flapjack.layout_print(fig, width=3.3, height=1., font_size=6.)
fig

In [ ]:
fname = 'Figure8C_CFP.png'
io.write_image(fig, fname)

### YFP

In [ ]:
fig = fj.plot(study=study.id,
             vector=vector_yfp_ids,
             signal=yfp_id,
             type='Velocity',
             pre_smoothing=21,
             post_smoothing=21,
             normalize='Mean/std',
             subplots='Media',
             markers='Vector',
             plot='Mean'
             )
fig

In [ ]:
fig = flapjack.layout_print(fig, width=3.3, height=1, font_size=6.)
fig

In [ ]:
fname = 'Figure8C_YFP.png'
io.write_image(fig, fname)

## Figure 8D - Analysing reaction velocity
To analyse the magnitude of gene expression, we can get the maximum reaction velocity over each time series from the Flapjack API:

In [ ]:
study = fj.get('study', name='CFPS of CFP YFP variants')
df = fj.analysis(study=study.id,
                 type='Max Velocity',
                 pre_smoothing=11,
                 post_smoothing=11
                )

Now we can group this dataframe to make a bar graph showing the maximum velocity in each of the combinations of extract batch and DNA that were tested:

In [ ]:
cols = {
    'E1': '#1f77b4', 
    'E2': '#ff7f0e', 
    'E3': '#2ca02c'
}
subplot_titles = ['CFP channel', 'YFP channel']
fig = make_subplots(rows=2, 
                    cols=1, 
                    vertical_spacing=0.15, 
                    horizontal_spacing=0.1,
                    subplot_titles=subplot_titles
                   )
row = 1
for name,name_data in df.groupby('Signal'):
    print(name)
    for media,media_data in name_data.groupby('Media'):
        grouped_dna = media_data.groupby('Vector')
        x = [dna for dna,data in grouped_dna]
        y = [data['Velocity'].mean() for dna,data in grouped_dna]        
        err_y = [data['Velocity'].std() for dna,data in grouped_dna]        
        error_y = dict(type='data', # value of error bar given in data coordinates
                        array=err_y,
                        visible=True)
        bar = go.Bar(x=x, 
                     y=y,
                     error_y=error_y, 
                     name='Batch '+media, 
                     legendgroup=media,
                     marker=dict(color=cols[media], line=dict(color='rgb(0,0,0)', width=1.)),
                     showlegend=row==1
                    )

        fig.add_trace(bar, row=row, col=1)
    row += 1

fig.update_yaxes(title_text='Max. velocity', col=1)
fig = layout_print(fig, width=3.3, height=3.3, font_size=6.)
fig.update_layout(barmode='group')
fig.show()
    

In [ ]:
fname = 'Figure8D.png'
io.write_image(fig, fname)